<a href="https://colab.research.google.com/github/biggity2bit/colab/blob/master/bernt_politifact_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [0]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [3]:
# Basic libraries
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Natural Language Processing
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://news-detector-project.s3.us-east-2.amazonaws.com/FakeNews.csv"
spark.sparkContext.addFile(url)
df_fake = spark.read.csv(SparkFiles.get("FakeNews.csv"), sep=",", header=True)

# Show DataFrame
df_fake.show()

+--------------------+--------------------+--------------+-----------+
|               title|         articletext|articlesubject|articledate|
+--------------------+--------------------+--------------+-----------+
| Donald Trump Sen...|Donald Trump just...|          News|  31-Dec-17|
| Drunk Bragging T...|House Intelligenc...|          News|  31-Dec-17|
| Sheriff David Cl...|On Friday, it was...|          News|  30-Dec-17|
| Trump Is So Obse...|On Christmas day,...|          News|  29-Dec-17|
| Pope Francis Jus...|Pope Francis used...|          News|  25-Dec-17|
| Racist Alabama C...|The number of cas...|          News|  25-Dec-17|
| Fresh Off The Go...|Donald Trump spen...|          News|  23-Dec-17|
| Trump Said Some ...|In the wake of ye...|          News|  23-Dec-17|
| Former CIA Direc...|Many people have ...|          News|  22-Dec-17|
| WATCH: Brand-New...|Just when you mig...|          News|  21-Dec-17|
| Papa John’s Foun...|A centerpiece of ...|          News|  21-Dec-17|
| WATC

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://news-detector-project.s3.us-east-2.amazonaws.com/TrueNews.csv"
spark.sparkContext.addFile(url)
df_true = spark.read.csv(SparkFiles.get("TrueNews.csv"), sep=",", header=True)

# Show DataFrame
df_true.show()

+--------------------+--------------------+--------------------+------------+
|               title|         articletext|      articlesubject| articledate|
+--------------------+--------------------+--------------------+------------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|   31-Dec-17|
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|   29-Dec-17|
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|   31-Dec-17|
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|   30-Dec-17|
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|   29-Dec-17|
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|   29-Dec-17|
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|   29-Dec-17|
|Factbox: Trump on...|The following sta...|        politicsNews|   29-Dec-17|
|Trump on Twitter ...|The following sta...|        politicsNews|   29-Dec-17|
|Alabama official ...|WASHINGTON (Reute...|        politicsNews|

In [7]:
# import pandas as pd
fake_df = df_fake.toPandas() 
true_df = df_true.toPandas() 
true_df.head()

,title,articletext,articlesubject,articledate
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,31-Dec-17
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,29-Dec-17
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,31-Dec-17
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,30-Dec-17
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,29-Dec-17
